In [1]:
import pandas as pd
import numpy as np
import nltk
from collections import defaultdict
from collections import Counter
from src.classes.database.offer import MongoOffer
from tqdm import tqdm
from enum import Enum
import json

tokenizer = nltk.RegexpTokenizer(r'\w+')

In [38]:
token_list = [tokenizer.tokenize(x.selling) for x in MongoOffer.objects()]
tagged = nltk.pos_tag(token_list[0])

tag = Enum('tag', ['BRAND'])

In [3]:
def get_all_words(token_list: list[list]) -> list:
    words = []
    for sentence in token_list:
        for word in sentence:
            words.append(word)
    return sorted(set(words))

def next_word_occurrence(key: str, look_ahead: int, tokens: list[list]) -> dict:
    counter = defaultdict(int)
    for sentence in tokens:
        for x in range(0, len(sentence)):
            if key == sentence[x]: #if we find the key in a sentence
                if (x+look_ahead) < len(sentence):
                    counter[sentence[x+look_ahead]] += 1
    return counter

def p_next_word(given: str, looking: str, look_ahead: int, tokens: list[list]) -> float:
    nwo: dict = next_word_occurrence(given, look_ahead, tokens)
    return nwo[looking] / sum(nwo.values()) if nwo[looking] != 0 else 0

def calculate_all_probabilities(tokens: list[list], look_ahead) -> dict:
    all_words = get_all_words(tokens)

    #Generating a list of all words
    word_canvas = defaultdict()
    for word in all_words:
        word_canvas[word] = None

    p_word = word_canvas.copy()
    for word in tqdm(p_word):
        p_word[word] = [defaultdict(float) for x in range(0,look_ahead)]
        for x in range(0,look_ahead):
            for sub_word in all_words:
                p = p_next_word(word, sub_word, x+1, tokens)
                if p > 0:
                    p_word[word][x][sub_word] = p_next_word(word, sub_word, x+1, tokens)
                #print(f"{word} | {str(x)} | {sub_word}: {p_word[word][x][sub_word]}")
    return p_word


In [63]:
#data = calculate_all_probabilities(token_list, 5)

  0%|          | 8/1975 [00:18<1:16:17,  2.33s/it]

Unexpected exception formatting exception. Falling back to standard exception



Traceback (most recent call last):
  File "/Users/donovanwright/opt/miniconda3/envs/HardwareScrape/lib/python3.11/site-packages/IPython/core/interactiveshell.py", line 3460, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "/var/folders/wy/h6fmhjrs0q7dr5n0vk4txs8r0000gn/T/ipykernel_37583/3664844784.py", line 1, in <module>
    data = calculate_all_probabilities(token_list, 5)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/var/folders/wy/h6fmhjrs0q7dr5n0vk4txs8r0000gn/T/ipykernel_37583/2648100920.py", line 34, in calculate_all_probabilities
    p = p_next_word(word, sub_word, x+1, tokens)
        ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/var/folders/wy/h6fmhjrs0q7dr5n0vk4txs8r0000gn/T/ipykernel_37583/2648100920.py", line 18, in p_next_word
    nwo: dict = next_word_occurrence(given, look_ahead, tokens)
                ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/var/folders/wy/h6fmhjrs0q7dr5n0vk4txs8r0000gn/T/ipykernel_37583/2

p_of_next_word['word']['looking_for_word'][INDEX HERE OF LOOKAHEAD]
return object like {occurrences: 1, probability: 0.5}

word1
    looking_for1
        0: {occurenecs:1, probability: 0.5}
        1:  {occurenecs:1, probability: 0.5}
        2:  {occurenecs:0, probability: 5}
    looking_for2

In [69]:
class HMatrix:
    def __init__(self) -> None:
        self.labels = None
        self.reverse_labels = None
        self.order = 0
        self.matrix = None

    def create_matrix(self, tokens: list[list], order: int):
        #Setup
        self.labels = get_all_words(token_list)
        self.reverse_labels = {self.labels[x]: x for x in range(0, len(self.labels))}
        self.order = order
        self.matrix = np.zeros((order, len(self.labels), len(self.labels)))

        #Iteration
        for sentence in tqdm(tokens):
            for x in range(0, len(sentence)):
                word = sentence[x]
                print(f"Word: {word} ({str(self.reverse_labels[word])})")
                for y in range(0, self.order):
                    if x+y+1 < len(sentence):
                        print(f"Lookahead {str(y+1)}: {sentence[x+y+1]} ({str(self.reverse_labels[sentence[x+y+1]])})")
                        #print(f"Wring to square: {str(y)}, writing to row: {str(self.reverse_labels[word])}"
                              #f", writing to column: {str(self.reverse_labels[sentence[x+y+1]])}")
                        self.matrix[y][self.reverse_labels[word]][self.reverse_labels[sentence[x+y+1]]] += 1

In [77]:
test = HMatrix()
test.create_matrix(token_list, 1)

100%|██████████| 1107/1107 [00:00<00:00, 45441.24it/s]

Word: logitech (1151)
Lookahead 1: g (896)
Word: g (896)
Lookahead 1: pro (1427)
Word: pro (1427)
Lookahead 1: x (1897)
Word: x (1897)
Lookahead 1: superlight (1688)
Word: superlight (1688)
Word: nintendo (1287)
Lookahead 1: switch (1700)
Word: switch (1700)
Lookahead 1: lite (1143)
Word: lite (1143)
Lookahead 1: gray (961)
Word: gray (961)
Lookahead 1: with (1887)
Word: with (1887)
Lookahead 1: original (1339)
Word: original (1339)
Lookahead 1: box (591)
Word: box (591)
Lookahead 1: extras (832)
Word: extras (832)
Lookahead 1: anbernic (501)
Word: anbernic (501)
Lookahead 1: rg351p (1508)
Word: rg351p (1508)
Word: samsung (1563)
Lookahead 1: 870 (408)
Word: 870 (408)
Lookahead 1: qvo (1466)
Word: qvo (1466)
Lookahead 1: 4tb (281)
Word: 4tb (281)
Word: evga (822)
Lookahead 1: rtx (1536)
Word: rtx (1536)
Lookahead 1: 3080 (204)
Word: 3080 (204)
Lookahead 1: xc3 (1926)
Word: xc3 (1926)
Lookahead 1: ultra (1794)
Word: ultra (1794)
Word: paypal (1371)
Lookahead 1: local (1147)
Word: local 

In [78]:
test.matrix[0][1151][896]

5.0

In [75]:
test.reverse_labels['controller']

688

In [ ]:
# test.matrix[0][1151][896] / sum (test.matrix[0][1151]) = probability that 1st order